## 匯入套件和資料

In [2]:
import os
import jieba
import numpy as np
import csv

In [3]:
def csv_create(file_name,model,co_id,list_name):
    with open(file_name,model,newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['公司名稱','95,96年','96,97年','97,98年','98,99年','99,100年','100,101年','101,102年','102,103年','103,104年','104,105年','105,106年','106,107年','107,108年','108,109年'])
        writer.writerow([co_id]+list_name)

In [4]:
all_file = []
txt_root = 'C:\\Users\\DC\\中文詞向量\\1909文字'

for root, dirs, files in os.walk(txt_root): 
    for file in files: 
        print(root + '\\' + file)
        all_file.append(root + '\\' + file)

C:\Users\DC\中文詞向量\1909文字\2006.txt
C:\Users\DC\中文詞向量\1909文字\2007.txt
C:\Users\DC\中文詞向量\1909文字\2008.txt
C:\Users\DC\中文詞向量\1909文字\2009.txt
C:\Users\DC\中文詞向量\1909文字\2010.txt
C:\Users\DC\中文詞向量\1909文字\2011.txt
C:\Users\DC\中文詞向量\1909文字\2012.txt
C:\Users\DC\中文詞向量\1909文字\2013.txt
C:\Users\DC\中文詞向量\1909文字\2014.txt
C:\Users\DC\中文詞向量\1909文字\2015.txt
C:\Users\DC\中文詞向量\1909文字\2016.txt
C:\Users\DC\中文詞向量\1909文字\2017.txt
C:\Users\DC\中文詞向量\1909文字\2018.txt
C:\Users\DC\中文詞向量\1909文字\2019.txt
C:\Users\DC\中文詞向量\1909文字\2020.txt


In [5]:
all_txt = []

for i in range(0,len(all_file)):
    txt = open(all_file[i], 'r', encoding='utf-8').read()
    txt = txt.replace('■','').replace('、','').replace('：','').replace('\n','').replace(' ','').replace('●','').replace('，','').replace('│','').replace('。','').replace(',','').replace(';','').replace('(','').replace(')','').replace('；','')
    a =jieba.lcut(txt)
    all_txt.append(a)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\DC\AppData\Local\Temp\jieba.cache
Loading model cost 0.432 seconds.
Prefix dict has been built successfully.


## 載入模型後將文字轉為向量！

In [6]:
from gensim.models import word2vec
model = word2vec.Word2Vec.load('word2vec.model') #載入模型

In [7]:
padding = np.zeros((model.vector_size,), dtype=np.float32) #0向量
all_vec_add = []
vec_add = 0

for j in range(0,len(all_txt)):
    for i in all_txt[j]:
        try:
            vec_add += model.wv[i]
        except:
            model.wv[i] = padding
            vec_add += model.wv[i]
    all_vec_add.append(vec_add)
    vec_add = 0
    
print(all_vec_add[1])

[ -14.0767565    46.24646      47.53234      19.766071    -34.680576
   43.732487     12.477381    147.3707      -83.32169     108.468124
   47.603924     13.609841   -198.20784     -57.676517    -55.061226
  -75.510826   -179.09776     -13.3747015   103.2035     -106.183685
   -6.0084286   -11.112196     23.652372    180.90865    -149.91566
  -63.548393    -70.273964    -28.19314     -34.56816     -20.61387
    8.137152     41.019234   -134.65855      49.59355     126.37415
  -67.87368    -181.58687    -105.995224    -45.851757     44.116825
 -110.09431     197.76761    -187.88338    -103.735344     21.7894
  -17.6429      -88.83157     -67.48575       0.93996143   24.815804
  168.5646      -71.89553    -264.7222       80.78578     -86.896515
 -143.5951      -92.90249     -27.388067      6.469459    195.60455
  -46.246902    -77.72149      70.49249    -219.9212     -174.51744
 -113.69289       5.977224      4.9034624    77.94604     -30.12458
   93.49504     -60.85464       8.0181465 

## 歐式計算距離寫入csv

In [8]:
#歐式
Euc = []

for i in range(0,len(all_vec_add)-1):
    x = all_vec_add[i]
    y = all_vec_add[i+1]
    dis = np.linalg.norm( all_vec_add[i] - all_vec_add[i+1])
    Euc.append(dis)
    print(dis)

57.314503
47.251766
299.04526
51.25886
62.751392
30.821426
376.77118
1812.0695
129.17519
384.53027
152.25471
191.33087
178.14421
303.39337


In [45]:
csv_file = 'euc_per_year.csv'
write_mode = 'w'
co_id = '1909'

csv_create(csv_file,write_mode,co_id,Euc)

## 餘弦計算距離寫入csv

In [9]:
#餘弦
Man = []

for i in range(0,len(all_vec_add)-1):
    x = all_vec_add[i]
    y = all_vec_add[i+1]
    
    #小數方式
    #np.set_printoptions(precision=10,suppress=True) #位數
    #dis =  np.array([1 - np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y))])
    
    #科學記號方式
    dis =  1 - np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y))
    print(dis)
    Man.append(dis)

1.0728836059570312e-06
1.2516975402832031e-06
4.0531158447265625e-06
2.3245811462402344e-06
1.5497207641601562e-06
1.7285346984863281e-06
2.86102294921875e-06
2.5033950805664062e-06
8.940696716308594e-07
5.364418029785156e-07
1.1920928955078125e-07
7.152557373046875e-07
3.5762786865234375e-07
2.980232238769531e-07


In [32]:
csv_file = 'man_per_year.csv'
write_mode = 'w'
co_id = '1909'

csv_create(csv_file,write_mode,co_id,Man)

### 其他

In [75]:
similarity = model.wv.similarity('股東',all_txt[0][2]) #兩點距離
similarity

0.9796264

In [10]:
for item in model.wv.most_similar('營收'):
    print(item)

('年', 0.9995686411857605)
('新', 0.999567449092865)
('為', 0.9995562434196472)
('，', 0.9995313882827759)
('增加', 0.9995272159576416)
('億元', 0.9995265007019043)
('上', 0.9995236396789551)
('；', 0.9995115399360657)
('及', 0.9995063543319702)
('下', 0.9995036721229553)


C:\Users\DC\anaconda3\lib\site-packages\gensim\models\keyedvectors.py:783: RuntimeWarning: invalid value encountered in true_divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]


In [ ]:
#https://www.itread01.com/content/1548060671.html